In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import time
import helppd2sql

In [2]:
sentiment_values = pd.read_csv('../data/chelseapower/fixed_sentiment_values.csv')

In [3]:
proj_sent = sentiment_values[['hashtag', 'ol_score', 'ol_avg']]

In [4]:
# proj_sent.apply(lambda x: x.isnull().sum())

In [5]:
# COALESCE (ol_score, ol_avg) NOT SUPPORTED YET
proj_sent['ol_score'] = proj_sent['ol_score'].combine_first(
    proj_sent['ol_avg'])

<ipython-input-5-03a699e8aca2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proj_sent['ol_score'] = proj_sent['ol_score'].combine_first(


In [6]:
# proj_sent.apply(lambda x: x.isnull().sum())

In [7]:
# UNREACHABLE
dropna_sentiments = proj_sent.dropna(axis = 0, how ='any')

In [8]:
# UNREACHABLE
dropna_sentiments.rename(columns = {'ol_score':'sentiment_score'}, inplace = True)

E:\ssdapps\anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
user_track = pd.read_csv('../data/chelseapower/user_track_hashtag_timestamp.csv')

In [10]:
# explorative analysis - comment out for benchmark
# df2.apply(lambda x: x.isnull().sum())

In [11]:
dropna_user = user_track.dropna(subset=['hashtag'])

In [12]:
counts = dropna_user['track_id'].value_counts()
popular = counts[counts >= 50]
pop_user_list = popular.index

In [13]:
pop_user = dropna_user[dropna_user['track_id'].
                                       isin(pop_user_list)]

In [36]:
context_content_features = pd.read_csv(
    '../data/chelseapower/fixed_context_content_features.csv')

In [65]:
counts3 = context_content_features['track_id'].value_counts()
popular3 = counts3[counts3 >= 50]
pop_context_index = popular3.index

In [66]:
print(len(context_content_features))
print(len(pop_context_list))
print(len(counts3))

11614671
37447
346700


In [67]:
pop_context = context_content_features[context_content_features['track_id'].
                                       isin(pop_context_list)]

In [68]:
len(pop_context)

9143294

In [17]:
# set(pop_context.columns) - {'coordinates','id',
#    'place','geo', 'created_at','artist_id','tweet_lang'}

In [18]:
proj_context = pop_context.drop(
    columns=['coordinates','id','place','geo','artist_id','tweet_lang'])
helppd2sql.drop2select(proj_context)

'"instrumentalness", "liveness", "speechiness", "danceability", "valence", "loudness", "tempo", "acousticness", "energy", "mode", "key", "track_id", "created_at", "lang", "time_zone", "user_id"'

In [19]:
# proj_context.apply(lambda x: x.isnull().sum())

In [20]:
dropna_context = proj_context.dropna(subset=
                 ['instrumentalness', 'liveness', 'speechiness',
                  'danceability', 'valence', 'acousticness',
                  'energy', 'mode', 'key', 'time_zone', 'user_id'])


Reduced the dataset from 10,887,911 to **6,413,576** by dropping all null value rows.

In [21]:
english_context = dropna_context[dropna_context['lang'] == 'en']

In [22]:
# CTE rename_context
english_context = english_context.drop(columns=['lang'])
english_context['time_zone'].replace('Eastern Time (US & Canada)', 'Eastern Time', inplace=True)
english_context['time_zone'].replace('Central Time (US & Canada)', 'Central Time', inplace=True)
english_context['time_zone'].replace('Pacific Time (US & Canada)', 'Pacific Time', inplace=True)
english_context['time_zone'].replace('Mountain Time (US & Canada)', 'Mountain Time', inplace=True)
english_context['time_zone'].replace('Alaska', 'Alaska Time', inplace=True)
english_context['time_zone'].replace('Hawaii', 'Hawaii Time', inplace=True)
english_context['time_zone'].replace('Arizona', 'Mountain Time', inplace=True)
english_context['time_zone'].replace('America/Chicago', 'Central Time', inplace=True)
english_context['time_zone'].replace('America/New_York', 'Eastern Time', inplace=True)
english_context['time_zone'].replace('America/Los_Angeles', 'Pacific Time', inplace=True)
english_context['time_zone'].replace('America/Denver', 'Mountain Time', inplace=True)
english_context['time_zone'].replace('America/Detroit', 'Eastern Time', inplace=True)
helppd2sql.drop2select(english_context)

'"instrumentalness", "liveness", "speechiness", "danceability", "valence", "loudness", "tempo", "acousticness", "energy", "mode", "key", "track_id", "created_at", "time_zone", "user_id"'

In [23]:
usa_context = english_context[english_context['time_zone'].isin(
    ['Eastern Time', 'Central Time', 'Pacific Time',
     'Mountain Time', 'Alaska Time', 'Hawaii Time'])]

In [24]:
df23 = usa_context.merge(
    pop_user, on=['track_id','created_at','user_id'], how='inner')


In [25]:
df23_drop = df23.drop(columns=['created_at'])
helppd2sql.drop2select(df23_drop)

'"instrumentalness", "liveness", "speechiness", "danceability", "valence", "loudness", "tempo", "acousticness", "energy", "mode", "key", "track_id", "time_zone", "user_id", "hashtag"'

In [ ]:
# Unsupported because df1 is in Python
df123 = dropna_sentiments.merge(df23, on=['hashtag'], how='inner')

In [ ]:
# Unsupported because df123 is in Python
df123 = df123.drop(columns=['hashtag'])

In [75]:
# Unsupported
df123['mode'] = df123['mode'].astype('Int64')
df123['user_id'] = df123['user_id'].astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69428 entries, 0 to 69427
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hashtag           69428 non-null  object 
 1   sentiment_score   69428 non-null  float64
 2   ol_avg            69428 non-null  float64
 3   user_id           69428 non-null  object 
 4   track_id          69428 non-null  object 
 5   created_at        69428 non-null  object 
 6   instrumentalness  69428 non-null  float64
 7   liveness          69428 non-null  float64
 8   speechiness       69428 non-null  float64
 9   danceability      69428 non-null  float64
 10  valence           69428 non-null  float64
 11  loudness          69428 non-null  float64
 12  tempo             69428 non-null  float64
 13  acousticness      69428 non-null  float64
 14  energy            69428 non-null  float64
 15  mode              69428 non-null  Int64  
 16  key               69428 non-null  float6

In [72]:
isinstance(globals()['df23'], pd.DataFrame)

True

In [76]:
#Create new column sentiment that will be the predictor based on the sentiment_score values
df123['sentiment'] = np.where(df123['sentiment_score']>= 0.01, 1, 0)

hashtag            0
sentiment_score    0
ol_avg             0
user_id            0
track_id           0
created_at         0
dtype: int64

In [ ]:
# Reorder columns
df_mvp = df123[['sentiment','sentiment_score','user_id','track_id','time_zone','instrumentalness',
              'liveness','speechiness','danceability','valence','loudness','tempo','acousticness','energy','mode','key']]
df_mvp.info()

In [ ]:
report = helppd2sql.global_report()
report_df = pandas.DataFrame(report)
report_df.to_csv('pandas_report.csv', index=False)
df_mvp.to_csv('df_mvp_p.csv', index=False)
exit()

In [ ]:

# Create new dataset with only user_id, track_id and time_zone and the category variables
df_timezone = df_mvp.drop(['user_id','track_id','sentiment','sentiment_score','instrumentalness','liveness','speechiness',
                             'danceability','valence','loudness','tempo','acousticness','energy','mode','key'], axis=1)
df_timezone.head()


# One Hot Encode the category data set
one_hot_df = pd.get_dummies(df_timezone)
one_hot_df.head()


# Rename the columns
one_hot_df.columns = ['tz_Alaska_Time','tz_Central_Time','tz_Eastern_Time','tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time']
one_hot_df.head()


#Drop time_zone from MVP dataset
df_mvp = df_mvp.drop(["time_zone"], axis=1)


#Concatenate one hot encoded dataframe
df_mvp = pd.concat([df_mvp,one_hot_df], axis=1)
#df_mvp.head()
print(f"preprocessing: {time.time() - begin}")
begin = time.time()


# Define X and y
y = df_mvp['sentiment']
X = df_mvp.drop('sentiment', axis = 1)


# Normalizing the data prior to fitting the model.
x_feats = ['sentiment_score','instrumentalness','liveness','speechiness','danceability',
           'loudness','tempo','acousticness','energy','mode','key','valence','tz_Alaska_Time',
           'tz_Central_Time','tz_Eastern_Time','tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time']

X = pd.get_dummies(df_mvp[x_feats], drop_first=False)
y = df_mvp.sentiment
X.head()


# Splitting the data into train and test sets (automatically uses stratified sampling by labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)


logreg = LogisticRegression(fit_intercept = True, C=1e12)
model_log = logreg.fit(X_train, y_train)
model_log


logreg.predict_proba(X_train)



# How may times was the classifier correct for the training set?
logreg.score(X_train, y_train)



# How may times was the classifier correct for the test set?
logreg.score(X_test, y_test)

print(f"ml time: {time.time() - begin}")